In [1]:
import pandas as pd
import re
import ast
from fuzzywuzzy import fuzz, process
import Levenshtein

pd.set_option('max_columns', None)
pd.options.display.max_colwidth = 1000

In [2]:
df_lecturas = pd.read_csv('../data/processed/recipe_df_lecturas.csv')

df_recetasgratis = pd.read_csv('../data/processed/recipe_df_recetasgratis.csv')

In [3]:
recipe_df = pd.concat([df_lecturas, df_recetasgratis], ignore_index=True)

In [31]:
recipe_df.head(1)

,recipe_image_url,recipe_name,recipe_url,ingredients_list,time_preparation(min),gluten_free,egg_free,sucrose_fructose_free,low_sodium,lactose_free,main_category,num_ingredients,input,matched_ingred,num_matched,num_missing_ingred,missing_ingred,recipe_rate
0,https://content-recetas.lecturas.com/medio/2019/12/16/ensalada_nicoise_y_ensalada_del_perigord_c0d6d2e1_800x800.jpg,Ensalada niçoise,https://www.lecturas.com/recetas/ensalada-nicoise_8575.html,"[patatas pequeñas rojas, judías verdes, tomates maduros, espinacas baby, cebolla morada, perejil fresco picado, limón el zumo]",60.0,0,0,0,0,0,principal,12,"[huevos, pechiga de pollo, lentejas, pimiento, arroz, leche, mantequilla, harina, sal, azúcar, aceite, zanahorias, queso, bacon, lechuga, calabacín, atún]","[huevos, aceite de oliva virgen extra, pimienta, sal, atún en conserva]",5,7,"[patatas pequeñas rojas, judías verdes, tomates maduros, espinacas baby, cebolla morada, perejil fresco picado, limón el zumo]",1.4


In [5]:
def ingred_list(column):
    ingred_list = ast.literal_eval(column)
    return ingred_list

In [6]:
recipe_df["ingredients_list"] = recipe_df.apply(lambda x: ingred_list(x["ingredients_list"]), axis=1)

In [ ]:
def clean_ingredients(column):
    ingred_list = []
    for i in column:
        try:
            huevo_ratio = fuzz.WRatio("huevo", i)
            if huevo_ratio >= 86:
                ingred_list.remove(i)
                new_list.append("huevo")
            else:
                pass
        except:
            pass
    return ingred_list

In [7]:
recipe_df.columns

Index(['recipe_image_url', 'recipe_name', 'recipe_url', 'ingredients_list',
       'time_preparation(min)', 'gluten_free', 'egg_free',
       'sucrose_fructose_free', 'low_sodium', 'lactose_free', 'main_category',
       'num_ingredients'],
      dtype='object')

In [8]:
ingredients_df = recipe_df.ingredients_list.apply(pd.Series) \
    .merge(recipe_df, right_index = True, left_index = True) \
    .melt(id_vars = ['recipe_image_url', 'recipe_name', 'recipe_url', 'ingredients_list',
       'time_preparation(min)', 'gluten_free', 'egg_free',
       'sucrose_fructose_free', 'low_sodium', 'lactose_free', 'main_category',
       'num_ingredients'], value_name = "ingredient")

In [9]:
ingredients_df = ingredients_df[ingredients_df['ingredient'].notna()]

In [10]:
ingredients_df.head()

,recipe_image_url,recipe_name,recipe_url,ingredients_list,time_preparation(min),gluten_free,egg_free,sucrose_fructose_free,low_sodium,lactose_free,main_category,num_ingredients,variable,ingredient
0,https://content-recetas.lecturas.com/medio/2019/12/16/ensalada_nicoise_y_ensalada_del_perigord_c0d6d2e1_800x800.jpg,Ensalada niçoise,https://www.lecturas.com/recetas/ensalada-nicoise_8575.html,"[patatas pequeñas rojas, huevos, judías verdes, tomates maduros, atún en conserva, espinacas baby, cebolla morada, perejil fresco picado, aceite de oliva virgen extra, limón el zumo, pimienta, sal]",60.0,0,0,0,0,0,principal,12,0,patatas pequeñas rojas
1,https://content-recetas.lecturas.com/medio/2019/07/17/ensalada_nicoise_y_ensalada_del_perigord_8e285865_800x800.jpg,Ensalada del périgord,https://www.lecturas.com/recetas/ensalada-perigord_8576.html,"[foie micuit, beicon gruesas, champiñones, berros, rúcula, tomates secos en aceite, piñones, jamón de pato, aceite de oliva]",25.0,0,0,0,0,0,principal,9,0,foie micuit
2,https://content-recetas.lecturas.com/medio/2020/03/05/xato_con_salsa_romesco_y_salpicon_de_marisco_d6b2df39_800x800.jpg,Xató con salsa romesco,https://www.lecturas.com/recetas/xato-salsa-romesco_8577.html,"[bacalao desalado, cebolleta, escarolas, aceitunas negras, anchoas en aceite, tomates maduros, cabeza de ajos, ñoras, pan seco, almendras peladas tostadas, avellanas peladas y tostadas, pimentón, sal, aceite de oliva, vinagre de vino blanco, guindillas]",35.0,0,0,0,0,0,principal,16,0,bacalao desalado
3,https://content-recetas.lecturas.com/medio/2019/07/25/ensalada_de_ceviche_de_langostinos_y_cebolla_y_ensalada_de_pollo_remolacha_y_manzana_ca9d87d5_800x800.jpg,Ensalada de ceviche de langostinos y cebolla,https://www.lecturas.com/recetas/ensalada-ceviche-langostinos-y-cebolla_8447.html,"[langostinos, pimienta, limas, tomates, cebolla morada, limones, guindilla, aguacate, cilantro, sal, lechugas brotes, germinados de remolacha]",25.0,0,0,0,0,0,principal,12,0,langostinos
4,https://content-recetas.lecturas.com/medio/2019/08/26/ensalada_de_ceviche_de_langostinos_y_cebolla_y_ensalada_de_pollo_remolacha_y_manzana_bf72acd4_800x800.jpg,"Ensalada de pollo, remolacha y manzana",https://www.lecturas.com/recetas/ensalada-pollo-remolacha-y-manzana_8448.html,"[pechugas de pollo, manzanas ácida, col lombarda, zanahorias, granada, remolacha, nueces peladas, pan rallado grueso, pimentón dulce, huevo, comino, semillas de sésamo negro, lechugas, aceite de oliva, sal, pimienta]",40.0,0,0,0,0,0,principal,16,0,pechugas de pollo


In [29]:
ingredients_df.to_csv (f'../data/processed/ingredients_df.csv', index=False)

In [11]:
input_ing = input('¿Qué tienes hoy en la nevera?')

¿Qué tienes hoy en la nevera? huevos, pechiga de pollo, lentejas, pimiento, arroz, leche, mantequilla, harina, sal, azúcar, aceite, zanahorias, queso, bacon, lechuga, calabacín, atún


In [12]:
input_ing

'huevos, pechiga de pollo, lentejas, pimiento, arroz, leche, mantequilla, harina, sal, azúcar, aceite, zanahorias, queso, bacon, lechuga, calabacín, atún'

In [13]:
def input_list(string):
    input_list = []
    str_split = string.split(", ")
    for x in str_split:
        input_list.append(x)
    return input_list

input_ingred = input_list(input_ing)

print(input_ingred)

['huevos', 'pechiga de pollo', 'lentejas', 'pimiento', 'arroz', 'leche', 'mantequilla', 'harina', 'sal', 'azúcar', 'aceite', 'zanahorias', 'queso', 'bacon', 'lechuga', 'calabacín', 'atún']


In [14]:
recipe_df["input"] = input_ing
recipe_df["input"] = recipe_df.apply(lambda x: input_list(x["input"]), axis=1)

In [15]:
def matched(ingred_list, input_ingred):
    new_list = []
    for element in input_ingred:
        try:
            best_match = process.extract(element, ingred_list, scorer=fuzz.WRatio, limit=1)
            if best_match[0][1] >= 86:
                ingred_list.remove(best_match[0][0])
                new_list.append(best_match[0][0])
            else:
                pass
        except:
            pass
    return new_list

In [16]:
recipe_df["matched_ingred"] = recipe_df.apply(lambda x: matched(x["ingredients_list"], x["input"])
                                                  , axis=1)

In [17]:
recipe_df.head()

,recipe_image_url,recipe_name,recipe_url,ingredients_list,time_preparation(min),gluten_free,egg_free,sucrose_fructose_free,low_sodium,lactose_free,main_category,num_ingredients,input,matched_ingred
0,https://content-recetas.lecturas.com/medio/2019/12/16/ensalada_nicoise_y_ensalada_del_perigord_c0d6d2e1_800x800.jpg,Ensalada niçoise,https://www.lecturas.com/recetas/ensalada-nicoise_8575.html,"[patatas pequeñas rojas, judías verdes, tomates maduros, espinacas baby, cebolla morada, perejil fresco picado, limón el zumo]",60.0,0,0,0,0,0,principal,12,"[huevos, pechiga de pollo, lentejas, pimiento, arroz, leche, mantequilla, harina, sal, azúcar, aceite, zanahorias, queso, bacon, lechuga, calabacín, atún]","[huevos, aceite de oliva virgen extra, pimienta, sal, atún en conserva]"
1,https://content-recetas.lecturas.com/medio/2019/07/17/ensalada_nicoise_y_ensalada_del_perigord_8e285865_800x800.jpg,Ensalada del périgord,https://www.lecturas.com/recetas/ensalada-perigord_8576.html,"[foie micuit, beicon gruesas, champiñones, berros, rúcula, piñones, jamón de pato, aceite de oliva]",25.0,0,0,0,0,0,principal,9,"[huevos, pechiga de pollo, lentejas, pimiento, arroz, leche, mantequilla, harina, sal, azúcar, aceite, zanahorias, queso, bacon, lechuga, calabacín, atún]",[tomates secos en aceite]
2,https://content-recetas.lecturas.com/medio/2020/03/05/xato_con_salsa_romesco_y_salpicon_de_marisco_d6b2df39_800x800.jpg,Xató con salsa romesco,https://www.lecturas.com/recetas/xato-salsa-romesco_8577.html,"[bacalao desalado, cebolleta, escarolas, aceitunas negras, tomates maduros, cabeza de ajos, ñoras, pan seco, almendras peladas tostadas, avellanas peladas y tostadas, pimentón, aceite de oliva, vinagre de vino blanco, guindillas]",35.0,0,0,0,0,0,principal,16,"[huevos, pechiga de pollo, lentejas, pimiento, arroz, leche, mantequilla, harina, sal, azúcar, aceite, zanahorias, queso, bacon, lechuga, calabacín, atún]","[sal, anchoas en aceite]"
3,https://content-recetas.lecturas.com/medio/2019/07/25/ensalada_de_ceviche_de_langostinos_y_cebolla_y_ensalada_de_pollo_remolacha_y_manzana_ca9d87d5_800x800.jpg,Ensalada de ceviche de langostinos y cebolla,https://www.lecturas.com/recetas/ensalada-ceviche-langostinos-y-cebolla_8447.html,"[langostinos, limas, tomates, cebolla morada, limones, guindilla, aguacate, cilantro, germinados de remolacha]",25.0,0,0,0,0,0,principal,12,"[huevos, pechiga de pollo, lentejas, pimiento, arroz, leche, mantequilla, harina, sal, azúcar, aceite, zanahorias, queso, bacon, lechuga, calabacín, atún]","[pimienta, sal, lechugas brotes]"
4,https://content-recetas.lecturas.com/medio/2019/08/26/ensalada_de_ceviche_de_langostinos_y_cebolla_y_ensalada_de_pollo_remolacha_y_manzana_bf72acd4_800x800.jpg,"Ensalada de pollo, remolacha y manzana",https://www.lecturas.com/recetas/ensalada-pollo-remolacha-y-manzana_8448.html,"[manzanas ácida, col lombarda, granada, remolacha, nueces peladas, pan rallado grueso, pimentón dulce, comino, semillas de sésamo negro]",40.0,0,0,0,0,0,principal,16,"[huevos, pechiga de pollo, lentejas, pimiento, arroz, leche, mantequilla, harina, sal, azúcar, aceite, zanahorias, queso, bacon, lechuga, calabacín, atún]","[huevo, pechugas de pollo, pimienta, sal, aceite de oliva, zanahorias, lechugas]"


In [18]:
def num_ingred(column):
    return len(column)

In [19]:
recipe_df["num_matched"] = recipe_df.apply(lambda x: num_ingred(x["matched_ingred"]), axis=1)

In [20]:
def num_missing(recipe_ingred, active_ingred):
    missing_ingred = recipe_ingred - active_ingred
    return missing_ingred


recipe_df["num_missing_ingred"] = recipe_df.apply(lambda x: num_missing(x["num_ingredients"],
                                                                        x["num_matched"]),axis=1)

In [21]:
def missing_ing(matched, ingred_list):
    missing_list = []
    for i in ingred_list:
        if i in matched:
            pass
        else:
            missing_list.append(i)
    return missing_list

In [22]:
recipe_df["missing_ingred"] = recipe_df.apply(lambda x: missing_ing(x["matched_ingred"],
                                                                        x["ingredients_list"]),axis=1)

In [23]:
def rating(match, missing):
    result = 0.70*match-0.30*missing
    return result

In [24]:
recipe_df["recipe_rate"] = recipe_df.apply(lambda x: rating(x["num_matched"], x["num_missing_ingred"]),
                                               axis=1)

In [25]:
filter_principal = recipe_df['main_category'] == "principal"
output_1 = recipe_df[filter_principal].sort_values(by=["recipe_rate", "num_missing_ingred"], 
                               ascending=[False, True]).head(10)

In [26]:
output_1

,recipe_image_url,recipe_name,recipe_url,ingredients_list,time_preparation(min),gluten_free,egg_free,sucrose_fructose_free,low_sodium,lactose_free,main_category,num_ingredients,input,matched_ingred,num_matched,num_missing_ingred,missing_ingred,recipe_rate
1909,https://content-recetas.lecturas.com/medio/2018/07/19/paso-a-paso-para-realizar-pechugas-villaroy-con-arandanos-resultado-final_f0a646e6_800x800.jpg,Pechugas villeroy con arándanos,https://www.lecturas.com/recetas/pechugas-villaroy-arandanos_5026.html,"[pan rallado, arándanos]",50.0,0,0,0,0,0,principal,11,"[huevos, pechiga de pollo, lentejas, pimiento, arroz, leche, mantequilla, harina, sal, azúcar, aceite, zanahorias, queso, bacon, lechuga, calabacín, atún]","[huevos, pechugas de pollo, pimienta, leche, mantequilla, harina, sal, aceite, lechugas variada]",9,2,"[pan rallado, arándanos]",5.7
1165,https://content-recetas.lecturas.com/medio/2018/07/19/paso-a-paso-para-hacer-canelones-de-verdura-y-pescado-resultado-final_830f39d3_800x800.jpg,Canelones de verdura y pescado,https://www.lecturas.com/recetas/canelones-verdura-y-pescado_3835.html,"[placas, puerros, diente de ajo, vino blanco, perejil]",50.0,0,0,1,0,0,principal,15,"[huevos, pechiga de pollo, lentejas, pimiento, arroz, leche, mantequilla, harina, sal, azúcar, aceite, zanahorias, queso, bacon, lechuga, calabacín, atún]","[pescado un resto de pescado asado, pimientos del piquillo, leche, mantequilla, harina, sal, aceite, zanahoria, queso parmesano rallado, calabacín]",10,5,"[placas, puerros, diente de ajo, vino blanco, perejil]",5.5
2150,https://content-recetas.lecturas.com/medio/2018/07/19/volovan-de-pollo-y-mozzarella_29492f85_800x800.jpg,Volován de pollo y mozzarella,https://www.lecturas.com/recetas/volovan-pollo-y-mozzarella_2476.html,"[volovanes, cebolla, nuez moscada]",35.0,0,0,0,0,0,principal,12,"[huevos, pechiga de pollo, lentejas, pimiento, arroz, leche, mantequilla, harina, sal, azúcar, aceite, zanahorias, queso, bacon, lechuga, calabacín, atún]","[huevos, pechugas de pollo, pimienta, leche, mantequilla, harina, sal, aceite de oliva, queso mozzarella bola]",9,3,"[volovanes, cebolla, nuez moscada]",5.4
18149,https://t2.rg.ltmcdn.com/es/images/1/7/0/img_berenjenas_rellenas_con_arroz_y_espinaca_52071_600.jpg,Berenjenas rellenas con arroz y espinaca,https://www.recetasgratis.net/receta-de-berenjenas-rellenas-con-arroz-y-espinaca-52071.html,"[berenjenas medianas, espinacas, cebolla pequeña, queso rallado]",45.0,0,0,0,0,0,principal,13,"[huevos, pechiga de pollo, lentejas, pimiento, arroz, leche, mantequilla, harina, sal, azúcar, aceite, zanahorias, queso, bacon, lechuga, calabacín, atún]","[pimienta, arroz, leche , mantequilla pequeño, harina, sal, aceite de oliva, zanahoria, queso]",9,4,"[berenjenas medianas, espinacas, cebolla pequeña, queso rallado]",5.1
440,https://content-recetas.lecturas.com/medio/2019/03/13/paso_a_paso_para_realizar_timbal_de_calabaza_patata_y_queso_resultado_final_978dbab7_800x800.jpg,"Timbal de calabaza, patata y queso",https://www.lecturas.com/recetas/timbal-calabaza-patata-y-queso_8149.html,"[calabaza de unos 600 g, patatas]",70.0,0,0,0,0,0,principal,10,"[huevos, pechiga de pollo, lentejas, pimiento, arroz, leche, mantequilla, harina, sal, azúcar, aceite, zanahorias, queso, bacon, lechuga, calabacín, atún]","[pimienta, leche, mantequilla, harina, sal, aceite, queso rallado, calabacín]",8,2,"[calabaza de unos 600 g, patatas]",5.0
2022,https://content-recetas.lecturas.com/medio/2018/07/19/pechugas-villeroy-con-setas_1d58c11e_800x800.jpg,Pechugas villeroy con setas,https://www.lecturas.com/recetas/pechugas-villeroy-setas_3544.html,"[setas, caldo de ave]",30.0,0,0,1,0,0,principal,10,"[huevos, pechiga de pollo, lentejas, pimiento, arroz, leche, mantequilla, harina, sal, azúcar, aceite, zanahorias, queso, bacon, lechuga, calabacín, atún]","[huevos, pechugas de pollo, pimienta, leche, mantequilla, harina, sal, aceite]",8,2,"[setas, caldo de ave]",5.0
2890,https://content-recetas.lecturas.com/

In [27]:
output_2 = recipe_df[filter_principal].sort_values(by=["num_missing_ingred", "recipe_rate"], 
                               ascending=[True, False]).head(10)

In [28]:
output_2

,recipe_image_url,recipe_name,recipe_url,ingredients_list,time_preparation(min),gluten_free,egg_free,sucrose_fructose_free,low_sodium,lactose_free,main_category,num_ingredients,input,matched_ingred,num_matched,num_missing_ingred,missing_ingred,recipe_rate
6943,https://t1.rg.ltmcdn.com/es/images/7/0/7/img_arroz_al_horno_con_queso_y_crema_58707_600.jpg,Arroz al horno con queso y crema,https://www.recetasgratis.net/receta-de-arroz-al-horno-con-queso-y-crema-58707.html,[],90.0,0,0,0,0,0,principal,7,"[huevos, pechiga de pollo, lentejas, pimiento, arroz, leche, mantequilla, harina, sal, azúcar, aceite, zanahorias, queso, bacon, lechuga, calabacín, atún]","[huevo, pimienta, arroz blanco cocido, leche o nata, mantequilla, sal, queso mozarella]",7,0,[],4.9
14914,https://t2.rg.ltmcdn.com/es/images/4/9/0/img_macarrones_gratinados_con_bechamel_53094_600.jpg,Macarrones gratinados con bechamel,https://www.recetasgratis.net/receta-de-macarrones-gratinados-con-bechamel-53094.html,[],NaN,0,0,0,0,0,principal,7,"[huevos, pechiga de pollo, lentejas, pimiento, arroz, leche, mantequilla, harina, sal, azúcar, aceite, zanahorias, queso, bacon, lechuga, calabacín, atún]","[una caja de macarrones con salsa de queso, pimienta, un vaso de leche, mantequilla, harina, sal, queso parmesano rallado]",7,0,[],4.9
11213,https://t2.rg.ltmcdn.com/es/images/9/1/0/img_tortilla_francesa_con_atun_y_queso_69019_600.jpg,Tortilla francesa con atún y queso,https://www.recetasgratis.net/receta-de-tortilla-francesa-con-atun-y-queso-69019.html,[],10.0,0,0,0,0,0,principal,5,"[huevos, pechiga de pollo, lentejas, pimiento, arroz, leche, mantequilla, harina, sal, azúcar, aceite, zanahorias, queso, bacon, lechuga, calabacín, atún]","[huevos, sal, aceite, queso rallado, atún]",5,0,[],3.5
15768,https://t1.rg.ltmcdn.com/es/images/4/6/3/img_salmon_a_la_mantequilla_dulce_59364_600.jpg,Salmón a la mantequilla dulce,https://www.recetasgratis.net/receta-de-salmon-a-la-mantequilla-dulce-59364.html,[],15.0,0,0,0,0,0,principal,5,"[huevos, pechiga de pollo, lentejas, pimiento, arroz, leche, mantequilla, harina, sal, azúcar, aceite, zanahorias, queso, bacon, lechuga, calabacín, atún]","[pimienta, mantequilla, salmón , azúcar, aceite de oliva]",5,0,[],3.5
2369,"data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7",Atún a la plancha,https://www.lecturas.com/recetas/atun-a-plancha_3291.html,[],NaN,0,0,0,0,0,principal,4,"[huevos, pechiga de pollo, lentejas, pimiento, arroz, leche, mantequilla, harina, sal, azúcar, aceite, zanahorias, queso, bacon, lechuga, calabacín, atún]","[pimienta, sal, aceite de oliva, lomos de atún]",4,0,[],2.8
6851,https://t1.rg.ltmcdn.com/es/images/3/1/2/img_pastel_de_arroz_salado_frio_62213_600.jpg,Pastel de arroz salado frío,https://www.recetasgratis.net/receta-de-pastel-de-arroz-salado-frio-62213.html,[],30.0,0,0,0,0,0,principal,4,"[huevos, pechiga de pollo, lentejas, pimiento, arroz, leche, mantequilla, harina, sal, azúcar, aceite, zanahorias, queso, bacon, lechuga, calabacín, atún]","[huevos, arroz, salsa rosa o mayonesa, atún]",4,0,[],2.8
11714,https://t2.rg.ltmcdn.com/es/images/6/7/2/img_lentejas_amarillas_con_arroz_caseras_14276_600.jpg,Lentejas amarillas con arroz caseras,https://www.recetasgratis.net/receta-de-lentejas-amarillas-con-arroz-caseras-14276.html,[],NaN,0,0,0,0,0,principal,4,"[huevos, pechiga de pollo, lentejas, pimiento, arroz, leche, mantequilla, harina, sal, azúcar, aceite, zanahorias, queso, bacon, lechuga, calabacín, atún]","[lentejas amarillas , arroz, sal, aceite]",4,0,[],2.8
14804,https://t1.rg.ltmcdn.com/es/images/3/0/5/img_masa_de_pizza_sin_levadura_57503_600.jpg,Masa de pizza sin levadura,https://www.recetasgratis.net/receta-de-masa-de-pizza-sin-levadura-57503.html,[],30.0,0,0,0,0,0,principal,4,"[huevos, pechiga de pollo, lentejas, pimiento, arroz, leche, mantequilla, harina, sal, azúcar, aceite, zanahorias, queso, bacon, lechuga, calabacín, atún]","[harina , sal, azúcar, aceite]",4,0,[],2.8
15156,https://t1.rg.lt